In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Первоначальная версия датасета содержит 17 полей со следующей информацией:

* hotel_address — адрес отеля;
* review_date — дата, когда рецензент разместил соответствующий отзыв;
* average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
* hotel_name — название отеля;
* reviewer_nationality — страна рецензента;
* negative_review — отрицательный отзыв, который рецензент дал отелю;
* review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
* positive_review — положительный отзыв, который рецензент дал отелю;
* review_total_positive_word_counts — общее количество слов в положительном отзыве.
* reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
* total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
* total_number_of_reviews — общее количество действительных отзывов об отеле;
* tags — теги, которые рецензент дал отелю;
* days_since_review — количество дней между датой проверки и датой очистки;
* additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
* lat — географическая широта отеля;
* lng — географическая долгота отеля.

In [3]:
hotels = pd.read_csv('hotels.csv')
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


**Проверим наличие пропусков и посмотрим типы данных:**

In [4]:
display(hotels.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

None

*Для корректной работы все данные в датафрейме, который будет использоваться при обучении модели, должны быть в числовых форматах int или float.*

*Есть ещё одно ограничение: в столбцах не должно быть пропущенных значений. Вместо каждого пропущенного значения нужно вычислить и поместить в ячейку максимально близкое к реальности значение.*

*Таким образом, чтобы код, отвечающий за обучение модели, заработал, необходимо использовать в нём датафрейм, содержащий только количественные признаки и не содержащий пропущенных значений.*

### Удалим столбцы, содержащие данные типов *object* и *string* и заполним пропущенные значения каким-то одним значением (*unknown*, нулём или средним арифметическим) для всего столбца.

In [14]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
hotels = hotels.fillna(0)

### Приступим к процессу обучения модели

1. **РАЗДЕЛЕНИЕ НАБОРА ДАННЫХ**

Прежде всего, для создания модели необходимо разделить датафрейм на набор данных, которые мы будем использовать для обучения модели, именуемый ***X***, и на целевую переменную — величину, значение которой мы будем предсказывать, ***y*** (в нашем случае это рейтинг отелей).

Далее каждый из полученных наборов мы делим на **тренировочный** (*train*, используется для обучения модели) и **тестовый** (*test*, используется для оценки точности модели). Такое деление осуществляется с помощью специального метода ```train_test_split()``` библиотеки *sklearn*. В параметрах метода (параметр ```test_size```) мы указываем, какую часть исходного датафрейма нужно оставить для тестирования модели. В нашем коде эта часть составляет 25 %, или 0.25.

Не существует идеального соотношения обучающей и тестовой выборки, однако принято выделять на часть обучения больше данных, около ⅔, а оставшуюся часть используют для проверки — ⅓ выборки. Таким образом, самые популярные значения параметра ```test_size``` составляют 0.3, 0.25, 0.2.

In [15]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [16]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [17]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

2. **СОЗДАНИЕ И ОБУЧЕНИЕ МОДЕЛИ**

Сам процесс создания и тестирования модели занимает всего четыре строчки кода. В качестве алгоритма мы будем использовать популярный и довольно мощный алгоритм ```RandomForestRegressor```. Он реализован в библиотеке *sklearn*.

In [18]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  

In [19]:
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)

In [20]:
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train) 

RandomForestRegressor()

In [21]:
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

3. **ОЦЕНКА КАЧЕСТВА МОДЕЛИ**

Для оценки качества модели — точности прогнозов, сделанных моделью, — мы будем использовать метрику (некий числовой показатель), которая называется **MAPE (mean absolute percentage error)**, средняя абсолютная процентная ошибка. Эту метрику очень легко интерпретировать. Если у вас *MAPE* = 11.4 %, то это говорит о том, что ошибка составила 11.4 % от фактических значений.

$$MAPE = 100 \% * {1/n}\sum\limits_{i=1}^n{|ytrue_i - ypred_i| \over |ytrue_i|},$$

где $ytrue_i$ — фактические значения прогноза, a $ypred_i$ — предсказанные.

Для расчета *MAPE* сравним предсказанные нашей моделью значения с реальными и посмотрим на значение ошибки.

In [22]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14143335420945896


Таким образом **MAPE (mean absolute percentage error)** средняя абсолютная процентная ошибка получилась равной 0.14143335420945896

### Теперь необходимо повысить точность прогнозов, которые делает модель, с помощью более тщательной подготовки данных.

Задачу, которая стоит далее, можно свести к пяти пунктам:

* **Удаление строковых значений.** Вам необходимо удалить из набора данных столбцы, данные в которых представлены не числами.
* **Очистка от пропущенных значений.** На предыдущем шаге мы делали это самым грубым из всех возможных способов, сейчас попробуйте подойти к процессу более гибко.
* **Создание новых признаков.** Мы попробуем создать новые столбцы с данными из существующих данных или с использованием внешних источников.
* **Преобразование признаков.** Применим различные преобразования над признаками вроде нормализации, стандартизации.
* **Отбор признаков.** Используем анализ мультиколлинеарности как шаг отбора признаков для модели.

**Основная задача** — при помощи этих шагов повысить качество предсказания. Чтобы понять, что качество улучшилось, необходимо ориентироваться на метрику *MAPE*: чем она меньше, тем лучше. Рассмотрим шаги подробнее.

**СТРОКОВЫЕ ДАННЫЕ**

В исходном наборе данных всего девять столбцов содержат числовые данные. Так что просто удалить все object-значения и считать задачу выполненной не получится. Чтобы улучшить качество модели, перед удалением необходимо основательно поработать со строковыми данными и извлечь из них как можно больше информации, которую можно представить в числовом виде.

**ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ**

В задачах по машинному обучению принято не удалять строки с пустыми значениями, а заполнять их максимально близкими к реальности данными. При помощи очистки данных и метода центральной тенденции необходимо проработать эти данные.

In [8]:
display(hotels.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

None

*Пропущенные значения содержаться в признаках lat и lng. Посмотрим для каких адресов отелей эти значения не заполнены.*

In [9]:
mark1 = hotels['lat'].isnull()
mark2 = hotels['lng'].isnull()
hotels[mark1 & mark2]['hotel_address'].value_counts()

hotel_address
Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria    496
Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria            421
W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria       285
4 rue de la P pini re 8th arr 75008 Paris France               224
Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria         182
W hringer Stra e 12 09 Alsergrund 1090 Vienna Austria          172
Savoyenstra e 2 16 Ottakring 1160 Vienna Austria               141
Bail n 4 6 Eixample 08010 Barcelona Spain                      113
Sieveringer Stra e 4 19 D bling 1190 Vienna Austria            112
Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria             63
23 Rue Damr mont 18th arr 75018 Paris France                    47
Hasenauerstra e 12 19 D bling 1190 Vienna Austria               44
Josefst dter Stra e 22 08 Josefstadt 1080 Vienna Austria        40
Paragonstra e 1 11 Simmering 1110 Vienna Austria                34
20 Rue De La Ga t 14th arr 75014 Paris France   

In [10]:
#выделим признак страны из адреса
hotels['country'] = hotels['hotel_address'].apply(
    lambda x: x.split()[-1] if x.split()[-1] != 'Kingdom'
    else 'UK')
hotels['country'].value_counts()

country
UK             196774
Spain           45132
France          44830
Netherlands     43006
Austria         29178
Italy           27883
Name: count, dtype: int64

In [11]:
#выделим признак города из адреса отеля
hotels['city'] = hotels['hotel_address'].apply(
    lambda x: x.split()[-2] if x.split()[-1] != 'Kingdom'
    else x.split()[-5])
hotels['city'].value_counts()

city
London       196774
Barcelona     45132
Paris         44830
Amsterdam     43006
Vienna        29178
Milan         27883
Name: count, dtype: int64

In [14]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 19 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [16]:
#расчитаем среднее значение 'lat','lng' для каждого города
median_country = hotels.groupby(by='city').median().loc[:,['lat','lng']]
median_country

TypeError: agg function failed [how->median,dtype->object]

**НОВЫЕ ПРИЗНАКИ**

Это, пожалуй, самая творческая часть работы на данном шаге. Создание новых признаков потребует способности вникать в контент, умения подключать к работе интуицию, творческого подхода и готовности к экспериментам. Необходимо извлечь несколько признаков из даты, проанализировать датасет на возможность добавления внешних источников данных.

**ПРЕОБРАЗОВАНИЕ ПРИЗНАКОВ**

Данные могут быть представлены в невыгодном для модели свете. Необходимо преобразовать данные, чтобы сделать их качественнее.

**ОТБОР ПРИЗНАКОВ**

Признаки могут дублировать информацию. **Анализ мультиколлинеарности** будем использовать, чтобы выбрать уникальные признаки для модели.